In [23]:
# Import Splinter, BeautifulSoup, Pandas and other dependencies
from splinter import Browser
from bs4 import BeautifulSoup as soup
import pandas as pd
from sqlalchemy import true
from webdriver_manager.chrome import ChromeDriverManager
import datetime as dt
import sys
import pandas as pd
import json
import time
import random

In [24]:
# Setup variable for news data url
categories_url = 'https://raw.githubusercontent.com/tcurlster/Fake-News-Recognition-and-Classification-Project/main/Resources/News_Category_Dataset_v2.json'

In [25]:
# Import news data with the links to the articles
news_categories = pd.read_json(categories_url, lines = True)
news_categories.head()

,category,headline,authors,link,short_description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26


In [39]:
# Sample the dataframe for testing scraping

# sample_dataframe = news_categories.groupby('category').apply(lambda x: x.sample(n=300)).reset_index(drop = True)
# sample_dataframe['category'].value_counts()

sample_dataframe = news_categories.sample(10)

ARTS              300
MONEY             300
PARENTS           300
POLITICS          300
QUEER VOICES      300
RELIGION          300
SCIENCE           300
SPORTS            300
STYLE             300
STYLE & BEAUTY    300
TASTE             300
TECH              300
THE WORLDPOST     300
TRAVEL            300
WEDDINGS          300
WEIRD NEWS        300
WELLNESS          300
WOMEN             300
WORLD NEWS        300
PARENTING         300
MEDIA             300
ARTS & CULTURE    300
LATINO VOICES     300
BLACK VOICES      300
BUSINESS          300
COLLEGE           300
COMEDY            300
CRIME             300
CULTURE & ARTS    300
DIVORCE           300
EDUCATION         300
ENTERTAINMENT     300
ENVIRONMENT       300
FIFTY             300
FOOD & DRINK      300
GOOD NEWS         300
GREEN             300
HEALTHY LIVING    300
HOME & LIVING     300
IMPACT            300
WORLDPOST         300
Name: category, dtype: int64

In [27]:
sample_dataframe.head()

,category,headline,authors,link,short_description,date
28061,HEALTHY LIVING,I Tracked My Sleep For A Month And All It Did ...,"Anna Sterling, HuffPost",https://www.huffingtonpost.com/entry/sleep-tra...,"For anxious high-achievers, adding one more me...",2017-04-28
25812,ARTS & CULTURE,Evan Pricco Curates 'What In The World' At Urb...,"Jaime Rojo & Steven Harrington, ContributorCo-...",https://www.huffingtonpost.com/entry/evan-pric...,"""The graffiti and Street Art movements – they ...",2017-05-24
115181,GREEN,"This Woodchuck Had A Bad Day, But It Got Better",Dominique Mosbergen,https://www.huffingtonpost.com/entry/woodchuck...,,2014-08-09
54169,TASTE,Stephen Colbert Loves Waffle House More Than You,Abigail Williams,https://www.huffingtonpost.com/entry/stephen-c...,He's the hilarious patron saint of waffles.,2016-07-06
22942,WOMEN,Why Did Contraception Stop Being Common Ground...,"Carole Joffe, Guest WriterGuest Writer",https://www.huffingtonpost.com/entry/why-did-c...,An administration firmly opposed to abortion i...,2017-06-27
81893,POLITICS,Shooting Suspect Sued Another Newsroom For Rac...,Dana Liebelson and Jessica Schulberg,https://www.huffingtonpost.com/entry/vester-fl...,Vester Flanagan said a co-worker at the other ...,2015-08-26
197992,FOOD & DRINK,Stylish Yellow Kitchen Accessories And Appliances,"Kitchen Daily, Kitchen Daily",https://www.huffingtonpost.com/entry/yellow-ki...,While spring might not be here quite yet -- th...,2012-02-28
112719,FIFTY,NBC's Tamron Hall On The Grandfather Who Was A...,,https://www.huffingtonpost.com/entry/grandpare...,,2014-09-06
184020,PARENTING,When Mom Is An Olympian,"Danielle Elliot, Contributor\nIndependent Docu...",https://www.huffingtonpost.com/entry/olympic-m...,I've always understood that women don't need t...,2012-07-27
75931,TASTE,5 Foolproof Muffins to Kick Off Baking Season,"Jennifer Segal, ContributorChef, Recipe Develo...",https://www.huffingtonpost.com/entry/5-foolpro...,Break out the apron (and the jeggings): baking...,2015-11-01


In [28]:
#Get iterable url list from the dataframe
sample_url_list = sample_dataframe['link'].tolist()

In [29]:
def get_browser():
    # Set up Splinter
    executable_path = {'executable_path': ChromeDriverManager().install()}
    return Browser('chrome', **executable_path, headless=False)

In [30]:
# Set up browser instance
browser = get_browser()



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [/Users/togneva/.wdm/drivers/chromedriver/mac64/100.0.4896.60/chromedriver] found in cache


In [31]:
%%time

# Create an empty list to hold the url data.
article_list = []

# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the urls in the list.
for i, url in enumerate(sample_url_list):

    # Group urls in sets of 10 for logging purposes.
    if (i % 10 == 0 and i >= 10):
        set_count += 1
        record_count = 1
        time.sleep(5)

    # Log the URL, record, and set numbers.
    print(f"Processing Record {record_count} of Set {set_count}")
    # Add 1 to the record count.
    record_count += 1
    
# Scrape each url in the list and append to the list of articles
    try:
        #Visit the news article site
        browser.visit(url)

        # Optional delay for loading the page
        browser.is_element_present_by_css('div.list_text', wait_time=1)

        # Convert the browser html to a soup object and then quit the browser
        html = browser.html
        news_soup = soup(html, 'html.parser')

        cli_article = ""
        cli_article_bites = news_soup.find_all('div', class_='primary-cli cli cli-text')
        for cli in cli_article_bites:
            cli_article+= cli.text + '\n'
            
            # Append the article to the list of articles
        article_list.append(cli_article)
    #         print(cli_article)
        
        
# If an error is experienced, skip the url and add a record of scraping error to the article list.
    except Exception:
        article_list.append('article could not be scraped')
        print(f"Url {url} can not be scraped. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1
Processing Record 2 of Set 1
Processing Record 3 of Set 1
Processing Record 4 of Set 1
Processing Record 5 of Set 1
Processing Record 6 of Set 1
Processing Record 7 of Set 1
Processing Record 8 of Set 1
Processing Record 9 of Set 1
Processing Record 10 of Set 1
-----------------------------
Data Retrieval Complete      
-----------------------------
CPU times: user 2.5 s, sys: 247 ms, total: 2.74 s
Wall time: 2min 15s


In [32]:
len(article_list)

10

In [33]:
sample_dataframe['article'] = article_list

In [34]:
article_list

['My first sleep log was five hours and four minutes. I was disappointed but also fascinated. My new Fitbit recorded how long I was asleep and how many times I was restless (19) ― just by wearing it to bed. But there was no ignoring my poor sleep habits. My bedtime read: 3:30 a.m. I was determined to do better.\nMost basic health advice, whether your goal is getting better skin, losing weight or feeling more productive at work, includes some variation on sleeping more. But despite clear guidelines ― adults should get seven to nine hours ― a lot of people miss the mark. According to Gallup, 40 percent of Americans get less than the recommended amount of sleep, up from 11 percent 70 years ago.\nI’m in the sleep-deprived camp. I know this not only because I rely on coffee to get me through the day, but also because my sleep tracker tells me.\nEven though technology is a big reason we’re sleeping less, we’re using it to help us solve the problem: Wearable devices let us set sleep goals and

In [35]:
sample_dataframe['article_length'] = sample_dataframe['article'].str.len()

In [36]:
sample_dataframe.head()

,category,headline,authors,link,short_description,date,article,article_length
28061,HEALTHY LIVING,I Tracked My Sleep For A Month And All It Did ...,"Anna Sterling, HuffPost",https://www.huffingtonpost.com/entry/sleep-tra...,"For anxious high-achievers, adding one more me...",2017-04-28,My first sleep log was five hours and four min...,4823
25812,ARTS & CULTURE,Evan Pricco Curates 'What In The World' At Urb...,"Jaime Rojo & Steven Harrington, ContributorCo-...",https://www.huffingtonpost.com/entry/evan-pric...,"""The graffiti and Street Art movements – they ...",2017-05-24,A new exhibition in Berlin’s neighborhood of S...,13008
115181,GREEN,"This Woodchuck Had A Bad Day, But It Got Better",Dominique Mosbergen,https://www.huffingtonpost.com/entry/woodchuck...,,2014-08-09,When a woodchuck needed a helping hand Friday ...,1078
54169,TASTE,Stephen Colbert Loves Waffle House More Than You,Abigail Williams,https://www.huffingtonpost.com/entry/stephen-c...,He's the hilarious patron saint of waffles.,2016-07-06,"When it comes to breakfast food, Stephen Colbe...",1491
22942,WOMEN,Why Did Contraception Stop Being Common Ground...,"Carole Joffe, Guest WriterGuest Writer",https://www.huffingtonpost.com/entry/why-did-c...,An administration firmly opposed to abortion i...,2017-06-27,Many Americans are puzzled by the all-out atta...,5787
